In [1]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell.

## Install dependencies
!apt-get install sox libsndfile1 ffmpeg
!pip install wget
!pip install text-unidecode

### Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2build1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox3 libwavpack1
Suggested packages:
  libsox-fmt-all
The following NEW packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base
  libsox3 libwavpack1 sox
0 upgraded, 7 newly installed, 0 to remove and 17 not upgraded.
Need to get 617 kB of archives.
After this operation, 1,764 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopencore-amrnb0 amd64 0.1.5-1 [94.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopencore-amrwb0 amd64 0.1.5-1 [49.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libsox3 am

In [2]:
### Install Hugging Face Hub
!python -m pip install huggingface_hub
!python -m pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.1 MB/s eta 0:00:00


# NeMo models on Hugging Face Hub

This guide will briefly show how to upload NeMo models to Hugging Face programmatically.

This enables community members to share their NeMo models (any model!) with all users of NeMo!

**Note**: While in this tutorial we showcase an ASR model, there is no particular restriction to any domain - all NeMo models (.nemo files) of every domain can be uploaded and shared in the same way.

# Login to Hugging Face

Use the notebook login, and access your user access token (or create one to upload models to Hugging Face).

For more information, visit the User Access Token section - https://huggingface.co/docs/hub/security-tokens

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
!git config --global credential.helper store

# Prepare a model to upload to HF

In this example, we will download a NeMo ASR model from NGC and then upload it to Hugging Face for simplicity and to showcase the method.

**You can swap out this ASR model for any model that you restore via `restore_from()` and follow the same steps to upload your own models !**

In [5]:
import torch
import torch.nn as nn

from omegaconf import DictConfig, OmegaConf, open_dict

In [6]:
import nemo.collections.asr as nemo_asr  # use any domain's models !
import nemo.collections.nlp as nemo_nlp  # use any domain's models !
import nemo.collections.tts as nemo_tts  # use any domain's models !

# Model Name

NeMo adheres to strict requirements when naming a model for upload to NGC / Hugging Face Hub.

It is **mandatory** to share the model name across the model card, the NeMo file itself. Otherwise NeMo model from Hugging Face will fail to restore correctly.

## Naming Convention

NeMo model names can vary based on domain and purpose. While we attempt to conform to standard guidelines when naming our models, we do not expect the same level of strictness for community contributions.

Here are some common guidelines we encourage (but do not enforce) users to follow :

- `Task name`: Usually a short 2-3 character representation of the task that the model performs.
  - `stt` = Speech To Text (ASR)
  - `tts` = Text to Speech (TTS)
  - `ssl` = (Speech) Self Supervised Learning (SSL)
  - `nlp` = Natural Language Processing (NLP)
  - `nmt` = Neural Machine Translation (NMT) and many more.

- `Language ID`: Usually a 2/3 digit universal language id. For multilingual models, each domain has its own rules, but some common ones are `{lang_1}{lang_2}{...}` or call them `multilingual.`

- `Model Identifier`: Since models vary so drastically across domains, there is a lot of flexibility here. We try to adhere to naming conventions in literature as much as possible. For example, you can attach `model architecture` (Conformer/Citrinet), `training loss` (CTC/Transducer), and `model size` (small, large, discrete integer sizes, etc.).

- `Optional: Additional Modifiers`: These are additional identifiers such as gender of speaker (TTS), dataset name (ls for Librispeech), etc. It can be set on a case-by-case basis.

All these name segments are jointed by `_`.

-----

As an example of the following model we will try today :

`{task name}_{language id}_{model identifier}_[OPTIONAL modifiers]` = `stt_en_conformer_ctc_small`

**Set the MODEL_NAME carefully** !

In [7]:
MODEL_NAME = "stt_en_conformer_ctc_small"

-----
**Restore a NeMo Model**

Here, we restore a model from NGC directly, but you can restore a model from your training runs using `restore_from()` or use a local .nemo file.

In [8]:
model = nemo_asr.models.ASRModel.from_pretrained(MODEL_NAME)

[NeMo I 2023-09-18 12:26:47 cloud:68] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_conformer_ctc_small/versions/1.6.0/files/stt_en_conformer_ctc_small.nemo to /root/.cache/torch/NeMo/NeMo_1.21.0rc0/stt_en_conformer_ctc_small/5d2d8e5b2b5adb8f5091363c6ba19c55/stt_en_conformer_ctc_small.nemo
[NeMo I 2023-09-18 12:26:49 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-09-18 12:26:50 mixins:170] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2023-09-18 12:26:50 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 64
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2023-09-18 12:26:50 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librispeech_withs

[NeMo I 2023-09-18 12:26:50 features:289] PADDING: 0
[NeMo I 2023-09-18 12:26:51 save_restore_connector:249] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_1.21.0rc0/stt_en_conformer_ctc_small/5d2d8e5b2b5adb8f5091363c6ba19c55/stt_en_conformer_ctc_small.nemo.


# Create a Hugging Face Model

Now that we have a NeMo model and have logged into Hugging Face with our user API key, we can begin by creating a new repository and uploading our model.

-----

After the model has been restored, create an HfApi object to interact with the model repository.

In [12]:
from huggingface_hub import HfApi
api = HfApi()
username = api.whoami()['name']


In [13]:
try:
  api.create_repo(repo_id=MODEL_NAME)
  print("Successfully created repository !")
except Exception as e:
  print("Repository is possibly already created. Refer to error here - \n\n", e)

Repository is possibly already created. Refer to error here - 

 403 Client Error: Forbidden for url: https://huggingface.co/api/repos/create (Request ID: Root=1-650844e8-10da963764e2b6c448a3bf25;dda96e08-8c65-4285-9102-77f335cbbb6d)

You don't have the rights to create a model under this namespace


In [14]:
from huggingface_hub import Repository

Note two essential names -

- `hf_model_name`: A string name that is the composite of your `username` and `MODEL_NAME` as set above. This name is used for multiple purposes, so keep track of it.

- `model_filename`: The actual filename of the NeMo model that will be uploaded to Hugging Face. Note that this filename is explicitly set to `{MODEL_NAME}.nemo`. If this model filename is altered, then the model cannot correctly be restored by NeMo when downloaded from Hugging Face Hub, so please be careful.

In [15]:
local_dir = f'model-{MODEL_NAME}/'
hf_model_name = f'{username}/{MODEL_NAME}'

commit_message = "Upload model"
model_filename = f'{MODEL_NAME}.nemo'

with Repository(local_dir=local_dir, clone_from=hf_model_name, repo_type='model').commit(commit_message):
  model.save_to(model_filename)

Cloning https://huggingface.co/omeres/stt_en_conformer_ctc_small into local empty directory.


OSError: ignored

In [16]:
print("Finished uploading model to :", hf_model_name)

Finished uploading model to : omeres/stt_en_conformer_ctc_small


## Test if the model works

Now that we uploaded the model, let's try to use it in NeMo !

The only change required between normally calling `from_pretrained(model_name)` is to call **`from_pretrained({username}/{filename})`**

In [19]:
hf_model_name = f'{username}/{MODEL_NAME}'
hf_model = nemo_asr.models.ASRModel.from_pretrained(hf_model_name)

RepositoryNotFoundError: ignored

In [18]:
print("Successfully used HF model -", hf_model_name)

Successfully used HF model - omeres/stt_en_conformer_ctc_small


# Model Card

Now that we have uploaded the model, we are nearly 50% done!

The next step is to update the model card to have some helpful information regarding the uploaded model and its scores compared to other models.

You can do this in two ways, manually (by clicking the link below) or programmatically fill in part of the model card by following the instructions below.

In [20]:
hf_url = f'https://huggingface.co/{username}/{MODEL_NAME}'
print(f"Visit {hf_url} to manually edit your model card")

Visit https://huggingface.co/omeres/stt_en_conformer_ctc_small to manually edit your model card


-----

Here, we are going to setup some variables for our model card.

First up are the tags:

In [21]:
TAGS = [
    "automatic-speech-recognition",  # Task id, refer to https://github.com/huggingface/datasets/blob/master/src/datasets/utils/resources/tasks.json for allowed values.
    "speech",  # add as many other tags as required
    "audio",
    "CTC",
    "Conformer",
    "Transformer",
    "NeMo",  # required for library identification
    "pytorch",  # required, for toolkit identification
    # "hf-asr-leaderboard",  # Should only be used if model is evaluated on benchmark scores for ASR.
]

-----

Next, we list down all the datasets that were used to train the model.

By convention, try to search if the dataset already exists on Hugging Face Datasets - it is usually listed at the top and in lower case.

If you train on datasets that don't yet exist in Hugging Face Datasets, you can still add them but try to differentiate them by using capitalized names.

In [22]:
# Replace all spaces with `-`
DATASETS = [
    "librispeech_asr",
    "mozilla-foundation/common_voice_7_0",
    "vctk",
    "fisher_corpus",
    "Switchboard-1",
    "WSJ-0",
    "WSJ-1",
    "National-Singapore-Corpus-Part-1",
    "National-Singapore-Corpus-Part-6",
    "VoxPopuli-(EN)",
    "Europarl-ASR-(EN)",
    "Multilingual-LibriSpeech-(2000-hours)",
]

-----

Now we create an automated template based on a config for the top portion of the readme file.

In [23]:
from dataclasses import dataclass, field
from typing import List, Optional, Dict, Any

@dataclass
class NeMoHuggingFaceModelConfig:
  language: List[str]
  license: str

  library_name: str = "nemo"
  datasets: List[str] = field(default_factory=lambda: DATASETS)
  thumbnail: Optional[str] = None
  tags: List[str] = field(default_factory=lambda: TAGS)
  model_index: Any = field(default_factory=lambda: [dict(name=MODEL_NAME, results=[])])

In [24]:
config = NeMoHuggingFaceModelConfig(language=['en'], license="cc-by-4.0")  # choose appropriate license here
config = OmegaConf.structured(config)

with open_dict(config):
  # Update `model_index` to `model-index`
  model_index = config.pop('model_index')
  config['model-index'] = model_index

  # Replace all spaces with `-` in datasets
  normalized_datasets = [ds_name.replace(" ", "-") for ds_name in config['datasets']]
  config['datasets'] = OmegaConf.create(normalized_datasets)

print(OmegaConf.to_yaml(config))

language:
- en
license: cc-by-4.0
library_name: nemo
datasets:
- librispeech_asr
- mozilla-foundation/common_voice_7_0
- vctk
- fisher_corpus
- Switchboard-1
- WSJ-0
- WSJ-1
- National-Singapore-Corpus-Part-1
- National-Singapore-Corpus-Part-6
- VoxPopuli-(EN)
- Europarl-ASR-(EN)
- Multilingual-LibriSpeech-(2000-hours)
thumbnail: null
tags:
- automatic-speech-recognition
- speech
- audio
- CTC
- Conformer
- Transformer
- NeMo
- pytorch
model-index:
- name: stt_en_conformer_ctc_small
  results: []



## Markdown Template

Now that we have an auto-generated header for our readme, next, we write down some template markdown for the actual contents of the markdown.

You can edit the code here directly if you want, or if you prefer the GUI to see the actual changes in real-time, you can finish uploading this model card and then edit the readme file on the Hugging Face webpage itself.

In [25]:
hf_model_name = f'{username}/{MODEL_NAME}'

TEMPLATE = f"""
## Model Overview

<DESCRIBE IN ONE LINE THE MODEL AND ITS USE>

## NVIDIA NeMo: Training

To train, fine-tune or play with the model you will need to install [NVIDIA NeMo](https://github.com/NVIDIA/NeMo). We recommend you install it after you've installed latest Pytorch version.
```
pip install nemo_toolkit['all']
```

## How to Use this Model

The model is available for use in the NeMo toolkit [3], and can be used as a pre-trained checkpoint for inference or for fine-tuning on another dataset.

### Automatically instantiate the model

```python
import nemo.collections.asr as nemo_asr
asr_model = nemo_asr.models.ASRModel.from_pretrained("{hf_model_name}")
```

### Transcribing using Python
First, let's get a sample
```
wget https://dldata-public.s3.us-east-2.amazonaws.com/2086-149220-0033.wav
```
Then simply do:
```
asr_model.transcribe(['2086-149220-0033.wav'])
```

### Transcribing many audio files

```shell
python [NEMO_GIT_FOLDER]/examples/asr/transcribe_speech.py \
 pretrained_name="{hf_model_name}" \
 audio_dir="<DIRECTORY CONTAINING AUDIO FILES>"
```

### Input

This model accepts 16000 KHz Mono-channel Audio (wav files) as input.

### Output

This model provides transcribed speech as a string for a given audio sample.

## Model Architecture

<ADD SOME INFORMATION ABOUT THE ARCHITECTURE>

## Training

<ADD INFORMATION ABOUT HOW THE MODEL WAS TRAINED - HOW MANY EPOCHS, AMOUNT OF COMPUTE ETC>

### Datasets

<LIST THE NAME AND SPLITS OF DATASETS USED TO TRAIN THIS MODEL (ALONG WITH LANGUAGE AND ANY ADDITIONAL INFORMATION)>

## Performance

<LIST THE SCORES OF THE MODEL -
      OR
USE THE Hugging Face Evaluate LiBRARY TO UPLOAD METRICS>

## Limitations

<DECLARE ANY POTENTIAL LIMITATIONS OF THE MODEL>

Eg:
Since this model was trained on publicly available speech datasets, the performance of this model might degrade for speech which includes technical terms, or vernacular that the model has not been trained on. The model might also perform worse for accented speech.


## References

<ADD ANY REFERENCES HERE AS NEEDED>

[1] [NVIDIA NeMo Toolkit](https://github.com/NVIDIA/NeMo)

"""

-----

Below, we will upload this model card in a temporary file called **`"readme_template.md"`**. This is done to prevent overwriting of the "final" model card that the user may have manually edited.

Once this step is finished, **please copy the contents of this file, create a README.md file and paste the contents into it**.

In [26]:
local_dir = f'model-{MODEL_NAME}/'
hf_model_name = f'{username}/{MODEL_NAME}'

commit_message = "Upload config"
filename = 'readme_template.md'

with Repository(local_dir=local_dir, clone_from=hf_model_name, repo_type='model').commit(commit_message):
  with open(filename, 'w') as f:
    f.write("---\n")
    f.write(OmegaConf.to_yaml(config))
    f.write("\n---\n\n")
    f.write(TEMPLATE)


Cloning https://huggingface.co/omeres/stt_en_conformer_ctc_small into local empty directory.


OSError: ignored

-----

Please visit the URL below to copy the contents of the `readme_template.md` file into your `README.md` file.

In [27]:
hf_url = f'https://huggingface.co/{username}/{MODEL_NAME}'
print(f"Visit {hf_url} to edit your model card from the generated template file `{filename}`")

Visit https://huggingface.co/omeres/stt_en_conformer_ctc_small to edit your model card from the generated template file `readme_template.md`


## Evaluation Results

Now that we have both the model checkpoint and the readme uploaded to the Hub, we can optionally add some evaluation results to the card as well!

While this next section is optional, it is highly encouraged to do!

In [30]:
import evaluate
evaluate.list_evaluation_modules(module_type='metric', with_details=True)

[{'name': 'precision', 'type': 'metric', 'community': False, 'likes': 1},
 {'name': 'code_eval', 'type': 'metric', 'community': False, 'likes': 11},
 {'name': 'roc_auc', 'type': 'metric', 'community': False, 'likes': 0},
 {'name': 'cuad', 'type': 'metric', 'community': False, 'likes': 0},
 {'name': 'xnli', 'type': 'metric', 'community': False, 'likes': 0},
 {'name': 'rouge', 'type': 'metric', 'community': False, 'likes': 24},
 {'name': 'pearsonr', 'type': 'metric', 'community': False, 'likes': 1},
 {'name': 'mse', 'type': 'metric', 'community': False, 'likes': 0},
 {'name': 'super_glue', 'type': 'metric', 'community': False, 'likes': 4},
 {'name': 'comet', 'type': 'metric', 'community': False, 'likes': 5},
 {'name': 'cer', 'type': 'metric', 'community': False, 'likes': 7},
 {'name': 'sacrebleu', 'type': 'metric', 'community': False, 'likes': 8},
 {'name': 'mahalanobis', 'type': 'metric', 'community': False, 'likes': 0},
 {'name': 'wer', 'type': 'metric', 'community': False, 'likes': 14

In [32]:
# Uncomment in order to see what values you can supply to the `evaluate` library to push to the Hub.
 #help(evaluate.push_to_hub)

In [33]:
hf_model_name = f'{username}/{MODEL_NAME}'
metric_value = 8.1  # value obtained from https://catalog.ngc.nvidia.com/orgs/nvidia/teams/nemo/models/stt_en_conformer_ctc_small

evaluate.push_to_hub(
    model_id=hf_model_name,
    task_type="automatic-speech-recognition",
    dataset_type="librispeech_asr",
    dataset_name="Librispeech (clean)",
    metric_type="wer",
    metric_name="WER",
    dataset_split="test",  # corresponds to test-clean set
    dataset_config="other",  # corresponds to test-clean set
    dataset_args=dict(language="en"),  # metadata for dataset
    # the actual score obtained by the model
    metric_value=metric_value,
)

ValueError: ignored

-----

Done! Now we have a model checkpoint, a model card as well as evaluation results all set up for the NeMo model on Hugging Face!

To add more metrics, you can copy-paste the above cell and repeat the procedure for as many metrics as needed!